In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
hit = pd.read_csv("Hitters.csv")
df =hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
y = df["Salary"]
x_ = df.drop(["Salary","League","Division","NewLeague"], axis = 1).astype("float64")
x = pd.concat([x_,dms[["League_N","Division_W","NewLeague_N"]]],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [4]:
#!pip install lightgbm

In [4]:
from lightgbm import LGBMRegressor

In [5]:
lgbm = LGBMRegressor()
lgbm_model = lgbm.fit(x_train,y_train)

# Tahmin

In [6]:
y_pred = lgbm_model.predict(x_test,num_iteration=lgbm_model.best_iteration_)

In [7]:
np.sqrt(mean_squared_error(y_test,y_pred))

363.8712087611089

# Model Tuning

In [8]:
lgbm_grid = {"learning_rate": [0.01,0.1,0.5,1],
            "n_estimators": [20,40,100,200,500,1000],
            "max_depth": [1,2,3,4,5,6,7,8],
            "colsample_bytree":[0.4,0.5,0.6,0.9,1]}

lgbm = LGBMRegressor()
lgbm_cv_model = GridSearchCV(lgbm,lgbm_grid,cv=10,n_jobs = -1,verbose=2)

In [9]:
lgbm_cv_model.fit(x_train,y_train)

Fitting 10 folds for each of 960 candidates, totalling 9600 fits


GridSearchCV(cv=10, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'colsample_bytree': [0.4, 0.5, 0.6, 0.9, 1],
                         'learning_rate': [0.01, 0.1, 0.5, 1],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8],
                         'n_estimators': [20, 40, 100, 200, 500, 1000]},
             verbose=2)

In [10]:
lgbm_cv_model.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 6,
 'n_estimators': 20}

In [11]:
lgbm_tuned = LGBMRegressor(colsample_bytree=0.5,learning_rate=0.1,max_depth=6,n_estimators=20)
lgbm_tuned.fit(x_train,y_train)

LGBMRegressor(colsample_bytree=0.5, max_depth=6, n_estimators=20)

In [13]:
y_pred =lgbm_tuned.predict(x_test)

In [14]:
np.sqrt(mean_squared_error(y_test,y_pred))

375.6085209015434